In [8]:
import pandas as pd
import re
from collections import Counter
import matplotlib.pyplot as plt
import squarify
from datetime import datetime
import numpy as np

In [2]:
#Load Data
causal = pd.read_csv('dh_causal_lookup.csv')
product= pd.read_csv('111.csv')
store = pd.read_csv('dh_store_lookup.csv')
transactions = pd.read_csv('dh_transactions.csv')

In [3]:
# รวมข้อมูลตาม upc, week, geography, store และหาผลรวมของ dollar_sales และ units
transactions_combine_by_upc_store_week = transactions.groupby(['upc', 'week', 'geography', 'store']).agg(
        total_dollar_sales=('dollar_sales', 'sum'),    # ผลรวมของ dollar_sales
        total_units=('units', 'sum')                   # ผลรวมของ units
    ).reset_index()
transactions_combine_by_upc_store_week = transactions_combine_by_upc_store_week.sort_values(by=['upc', 'week'])
transactions_combine_by_upc_store_week

,upc,week,geography,store,total_dollar_sales,total_units
0,111112360,13,1,182,5.59,1
1,111112360,23,1,22,4.99,1
2,111112360,25,1,164,5.99,1
3,111112360,26,1,139,5.99,1
4,111112360,27,1,75,5.79,1
...,...,...,...,...,...,...
3473098,9999985766,104,2,316,1.99,1
3473099,9999985766,104,2,348,1.99,1
3473100,9999985766,104,2,349,1.99,1
3473101,9999985766,104,2,354,1.99,1


In [4]:
# ทำการ merge transactions_str กับ causal_lookup โดยใช้คีย์ upc, week, geography, และ store
merged_transactions = transactions_combine_by_upc_store_week.merge(causal[['upc', 'week', 'geography', 'store', 'feature_desc']],
                                             on=['upc', 'week', 'geography', 'store'], how='left')


In [5]:
merged_transactions = merged_transactions.dropna(subset=['feature_desc'])
merged_transactions

,upc,week,geography,store,total_dollar_sales,total_units,feature_desc
5938,601011292,44,1,159,2.00,1,Not on Feature
6036,601011292,45,1,159,10.00,5,Not on Feature
6139,601011292,46,2,326,5.00,2,Not on Feature
6220,601011292,47,2,326,2.50,1,Not on Feature
6308,601011292,49,1,1,2.00,1,Interior Page Feature
...,...,...,...,...,...,...,...
3466910,9999985261,104,1,154,8.10,6,Not on Feature
3466919,9999985261,104,1,170,3.87,3,Not on Feature
3466920,9999985261,104,1,171,6.75,5,Not on Feature
3471357,9999985766,58,1,205,1.25,1,Not on Feature


In [15]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder

data = merged_transactions

# แปลงข้อมูล categorical (upc, geography, store, feature_desc) เป็นตัวเลข
label_encoder_upc = LabelEncoder()
label_encoder_geography = LabelEncoder()
label_encoder_store = LabelEncoder()
label_encoder_feature = LabelEncoder()

data['upc_encoded'] = label_encoder_upc.fit_transform(data['upc'])
data['geography_encoded'] = label_encoder_geography.fit_transform(data['geography'])
data['store_encoded'] = label_encoder_store.fit_transform(data['store'])
data['feature_encoded'] = label_encoder_feature.fit_transform(data['feature_desc'])

X = data[['upc_encoded', 'week', 'geography_encoded', 'store_encoded', 'total_units', 'feature_encoded']]
y = data['total_dollar_sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


xgboost_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5)
xgboost_model.fit(X_train, y_train)

y_pred = xgboost_model.predict(X_test)


mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")

comparison_df = pd.DataFrame({'True Sales': y_test, 'Predicted Sales': y_pred})
print(comparison_df.head(10))  
xgboost_model_d = xgboost_model

Mean Squared Error (MSE): 0.47929789878740997
Mean Absolute Error (MAE): 0.30152471407441217
         True Sales  Predicted Sales
1020944        5.97         5.280195
851010        10.50        10.399652
1164578        5.94         5.751088
10693          1.67         1.690502
2569493        2.00         2.025164
2859053        5.50         5.613140
1363200        2.00         2.016460
2527165        3.00         2.537106
824595        23.00        24.291695
1643818        5.01         5.417445


In [12]:
label_encoder_upc = LabelEncoder()
label_encoder_geography = LabelEncoder()
label_encoder_store = LabelEncoder()
label_encoder_feature = LabelEncoder()

data['upc_encoded'] = label_encoder_upc.fit_transform(data['upc'])
data['geography_encoded'] = label_encoder_geography.fit_transform(data['geography'])
data['store_encoded'] = label_encoder_store.fit_transform(data['store'])
data['feature_encoded'] = label_encoder_feature.fit_transform(data['feature_desc'])


X = data[['upc_encoded', 'week', 'geography_encoded', 'store_encoded', 'total_dollar_sales', 'feature_encoded']]
y = data['total_units']  


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xgboost_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5)


xgboost_model.fit(X_train, y_train)


y_pred = xgboost_model.predict(X_test)
y_pred_rounded = np.rint(y_pred).astype(int)  


mse = mean_squared_error(y_test, y_pred_rounded)
mae = mean_absolute_error(y_test, y_pred_rounded)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")

# แสดงผลการทำนายและค่าจริง
comparison_df = pd.DataFrame({'True Units': y_test, 'Predicted Units': y_pred_rounded})
print(comparison_df.head(10))  
xgboost_model_u = xgboost_model

Mean Squared Error (MSE): 0.7448319484776171
Mean Absolute Error (MAE): 0.18728616198499182
         True Units  Predicted Units
1020944           3                3
851010            7                7
1164578           6                6
10693             1                1
2569493           2                2
2859053          11               11
1363200           1                1
2527165           2                2
824595           25               34
1643818           3                3
